# Distgen example

Similar to the simple example, but generating particles with Distgen

In [ ]:
from distgen import Generator

In [ ]:
# Nicer plotting
import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams["figure.figsize"] = (8, 4)

In [ ]:
YAML = """
n_particle: 10000
random_type: hammersley
species: electron

start:
  type: cathode
  MTE:
    value: 414
    units: meV    

total_charge:
  value: 250
  units: pC
    
r_dist:
  n_sigma_cutoff: 1.5
  sigma_xy:
    value: 0.4
    units: mm
  type: radial_gaussian

t_dist:
  type: superposition
  dists: 
    d1: 
      type: gaussian
      avg_t:
        units: ps
        value: -1
      sigma_t:
        units: ps
        value: 1
    d2: 
      type: gaussian
      avg_t:
        units: ps
        value: 1
      sigma_t:
        units: ps
        value: 1
 

"""

G = Generator(YAML)

In [ ]:
# Tune the two dist separation
G["t_dist:dists:d1:avg_t:value"] = -1
G["t_dist:dists:d2:avg_t:value"] = 1
G.run()
GP = G.particles
GP.plot("t")
GP.plot("pz")

In [ ]:
from impact import Impact

import os

In [ ]:
ifile = "templates/lcls_injector/ImpactT.in"
os.path.exists(ifile)

In [ ]:
# Make Impact object
I = Impact(ifile, initial_particles=G.particles, verbose=True)

In [ ]:
# This will use the initial particles
I.write_initial_particles(update_header=True)

In [ ]:
# Change some things
I.header["Nx"] = 16
I.header["Ny"] = 16
I.header["Nz"] = 16
I.header["Dt"] = 5e-13

# Turn Space Charge off
I.header["Bcurr"] = 0

# Other switches
I.timeout = 1000
# Switches for MPI
I.numprocs = 0

In [ ]:
# Change stop location
I.stop = 1.5
# I.ele['stop_1']['s'] = I.ele['OTR2']['s']+.001

In [ ]:
I.run()

In [ ]:
I.input.keys()

In [ ]:
I.output.keys()

In [ ]:
I.output["stats"].keys()

In [ ]:
I.output["slice_info"].keys()

# Particles

In [ ]:
# Particles are automatically parsed in to openpmd-beamphysics ParticleGroup objects
I.output["particles"]

In [ ]:
PI = I.output["particles"]["initial_particles"]
PF = I.output["particles"]["final_particles"]

In [ ]:
# Original particles
GP.plot("t", "pz")

In [ ]:
# Readback of initial particles from Impact-T.
PI.plot("t", "pz")

In [ ]:
# The initial time was shifted to account for this
I.header["Tini"]

In [ ]:
# Get the final particles, calculate some statistic
P = I.output["particles"]["final_particles"]
P["mean_energy"]

In [ ]:
# Show the units
P.units("mean_energy")

In [ ]:
P.plot("z", "pz")

# Stats

In [ ]:
# Impact's own calculated statistics can be retieved
len(I.stat("norm_emit_x")), I.stat("norm_emit_x")[-1]

In [ ]:
# Compare these.
key1 = "mean_z"
key2 = "sigma_x"
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1 + f" ({units1})")
plt.ylabel(key2 + f" ({units2})")
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles],
    [I.particles[name][key2] for name in I.particles],
    color="red",
)

# Archive, and restart from the middle

In [ ]:
afile = I.archive()
I2 = Impact(verbose=False)
I2.load_archive(afile)

# Patch in these particles
I2.initial_particles = I2.particles["YAG02"]

# Turn off cathode start
I2.header["Flagimg"] = 0
I2.configure()

In [ ]:
# Run again
I2.use_mpi = True
I2.run()

In [ ]:
# Compare these.
key1 = "mean_z"
key2 = "sigma_x"
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1 + f" ({units1})")
plt.ylabel(key2 + f" ({units2})")
plt.plot(I.stat(key1), I.stat(key2), color="black", label="original run")
plt.plot(I2.stat(key1), I2.stat(key2), color="red", label="restart run")
plt.scatter(
    [I.particles[name][key1] for name in I.particles],
    [I.particles[name][key2] for name in I.particles],
    color="black",
)

plt.scatter(
    [I2.particles[name][key1] for name in I2.particles],
    [I2.particles[name][key2] for name in I2.particles],
    color="red",
    marker="x",
)
plt.legend()

In [ ]:
# Cleanup
os.remove(afile)